In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch 
import torchvision 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

training_data = datasets.FashionMNIST(
    root="data",
    train=True, 
    download=True, 
    transform=transform
)

validation_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transform
)

training_loader = DataLoader(training_data, batch_size=64, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=64, shuffle=True)

training_data = datasets.FashionMNIST(
    root="data",
    train=True, 
    download=True, 
    transform=transform
)

validation_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transform
)


In [13]:
'''
IMG SIZE = (W-F+2P)/S + 1
'''

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet,self).__init__()
        self.conv1 = nn.Sequential(
            ### SIZE : 1 * 227 * 227
            nn.Conv2d(in_channels = 1,out_channels = 96,kernel_size = 11, padding = 0, stride = 4),
            ### SIZE : 96 * 55 * 55
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),
            ### SIZE : 96 * 27 * 27
        )

        self.conv2 = nn.Sequential(
            ### SIZE : 96 * 27 * 27 
            nn.Conv2d(in_channels = 96,out_channels = 256,kernel_size = 5, padding = 2, stride = 1),
            ### SIZE = 256 * 27 * 27
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),
            ### SIZE : 256 * 13 * 13
        )

        self.conv3 = nn.Sequential(
            ### SIZE : 256 * 13 * 13
            nn.Conv2d(in_channels = 256, out_channels = 384, kernel_size = 3,padding = 1, stride = 1),
            nn.ReLU(),
            ### SIZE : 384 * 13 * 13
        )
        
        self.conv4 = nn.Sequential(
            ### SIZE : 384 * 13 * 13
            nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size = 3,padding = 1 , stride = 1),
            nn.ReLU(),
            ### SIZE : 384 * 13 * 13
        )

        self.conv5 = nn.Sequential(
            ### SIZE : 384 * 13 * 13
            nn.Conv2d(in_channels = 384, out_channels = 256, kernel_size = 3,padding = 1 , stride = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2)
            ### SIZE : 256 * 6 * 6        
        )

        self.fc1 = nn.Linear( 256 * 6 * 6, 4096 )
        self.fc2 = nn.Linear( 4096, 4096 )
        self.fc3 = nn.Linear( 4096, 10)
        
    def forward(self, x:torch.Tensor):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)


        x = x.view(x.size(0),-1)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x,0.5)
        x = self.fc2(x)
        x = F.relu(x)
        x = F.dropout(x,0.5)
        x = self.fc3(x)
        x = F.log_softmax(x,dim=1)

        return x


In [14]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
model = AlexNet().to(device)
criterion = F.nll_loss # nll_loss : negative log likelihood los
optimizer = optim.Adam(model.parameters()) # model(신경망) 파라미터를 optimizer에 전달해줄 때 nn.Module의 parameters() 메소드를 사
epochs = 10
batch_size = 512

In [15]:
from torchsummary import summary as summary_


summary_(model, (1,227,227), batch_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [512, 96, 55, 55]          11,712
              ReLU-2          [512, 96, 55, 55]               0
 LocalResponseNorm-3          [512, 96, 55, 55]               0
         MaxPool2d-4          [512, 96, 27, 27]               0
            Conv2d-5         [512, 256, 27, 27]         614,656
              ReLU-6         [512, 256, 27, 27]               0
 LocalResponseNorm-7         [512, 256, 27, 27]               0
         MaxPool2d-8         [512, 256, 13, 13]               0
            Conv2d-9         [512, 384, 13, 13]         885,120
             ReLU-10         [512, 384, 13, 13]               0
           Conv2d-11         [512, 384, 13, 13]       1,327,488
             ReLU-12         [512, 384, 13, 13]               0
           Conv2d-13         [512, 256, 13, 13]         884,992
             ReLU-14         [512, 256,

In [16]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # enumberate() : 인덱스와 원소로 이루어진 튜플(tuple)을 만들어줌
        target = target.type(torch.LongTensor)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # 항상 backpropagation 하기전에 미분(gradient)을 zero로 만들어주고 시작해야 한다.
        output = model(data)
        loss = criterion(output, target) # criterion = loss_fn
        loss.backward() # Computes the gradient of current tensor w.r.t. graph leaves
        optimizer.step() # step() : 파라미터를 업데이트함
        if (batch_idx + 1) % 30 == 0:
            print("Train Epoch:{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [17]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)  # -> mean
        print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
        print('='*50)

In [ ]:
for epoch in range(1, epochs+1):
    train(model, device, training_loader, optimizer, epoch)
    test(model, device, validation_loader)

Train Epoch:1 [1856/60000 (3%)]	Loss: 1.065234
Train Epoch:1 [3776/60000 (6%)]	Loss: 0.795542
Train Epoch:1 [5696/60000 (9%)]	Loss: 0.743207
Train Epoch:1 [7616/60000 (13%)]	Loss: 0.789146
Train Epoch:1 [9536/60000 (16%)]	Loss: 0.502621
Train Epoch:1 [11456/60000 (19%)]	Loss: 0.443591
Train Epoch:1 [13376/60000 (22%)]	Loss: 0.449500
Train Epoch:1 [15296/60000 (25%)]	Loss: 0.652632
Train Epoch:1 [17216/60000 (29%)]	Loss: 0.432769
Train Epoch:1 [19136/60000 (32%)]	Loss: 0.553776
Train Epoch:1 [21056/60000 (35%)]	Loss: 0.589020
Train Epoch:1 [22976/60000 (38%)]	Loss: 0.364813
Train Epoch:1 [24896/60000 (41%)]	Loss: 0.343011
Train Epoch:1 [26816/60000 (45%)]	Loss: 0.381429
Train Epoch:1 [28736/60000 (48%)]	Loss: 0.201259
Train Epoch:1 [30656/60000 (51%)]	Loss: 0.458894
Train Epoch:1 [32576/60000 (54%)]	Loss: 0.495144
Train Epoch:1 [34496/60000 (57%)]	Loss: 0.388077
Train Epoch:1 [36416/60000 (61%)]	Loss: 0.274467
Train Epoch:1 [38336/60000 (64%)]	Loss: 0.294548
Train Epoch:1 [40256/60000 (